<a href="https://colab.research.google.com/github/Fengrui-Liu/nad21_ictfi/blob/main/nad21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import xgboost as xgb
import catboost as cab
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler
import pickle
import gc
import time
import os
gc.enable()

New features generation.

In [2]:
def same_subnet(dst,src):
  dst_subnet = dst.split('.')[:-2]
  src_subnet = src.split('.')[:-2]
  if dst_subnet == src_subnet:
    return 1
  return 0

def dst_is_broadcastIp(ip):
  tail = ip.split('.')[-1]
  if tail == '255':
    return 1
  return 0

def subnet(ip):
  return str(ip.split('.')[0:3])

def net(ip):
  return int(ip.split('.')[3])

def checkport(spt,dpt):
  if spt ==0 and dpt==0:
    return 1
  return 0

def checkicmp(app):
  if app == 'icmp':
    return 1
  return 0

def checkprobe(group):
  n = len(group)   
  net_set = set(group['dst_net'])

  if abs((len(net_set)-1) - (max(net_set) - min(net_set)))<2 and len(net_set)>7 :
    group['cnt_probe'] = len(net_set)
    timeStart = time.strptime(group.iloc[0].time, "%Y-%m-%d %H:%M:%S")
    timeEnd = time.strptime(group.iloc[-1].time, "%Y-%m-%d %H:%M:%S")
    dens = len(net_set) / (int(time.mktime(timeEnd)) - int(time.mktime(timeStart)))
    group['is_probe'] =  0 if dens < 0.1 else 1

  else:
    group['cnt_probe'] = 0
    group['is_probe'] = 0

  group['cnt_dpt'] = len(set(group['dpt']))
  group['cnt_spt'] = len(set(group['spt']))
  group['cnt_dst_grouped'] = len(set(group['dst']))
  
  return group
 

Drop the uesless features

In [3]:
drop_list =['out (bytes)','in (bytes)','duration','app','sub_dst','dst_net']
feature_col = []

In [4]:
def trans_df(df, drop = True):

  df['is_broadcast'] = list(map(dst_is_broadcastIp, df['dst']))
  df['is_subnet'] = list(map(same_subnet,df['dst'],df['src']))
  df['sub_dst'] = list(map(subnet,df['dst']))
  df['dst_net'] = list(map(net,df['dst']))
  df['is_icmp'] = list(map(checkicmp,df['app']))
  df['is_same_port'] = list(map(checkport,df['spt'],df['dpt']))

  cnt_list = ['cnt_src', 'cnt_serv_dst','cnt_src_slow','cnt_serv_dst_slow', 'cnt_src_conn','cnt_serv_dst_conn']


  flat_group_id = df.groupby(['dst'])
  for feature in cnt_list:
    df[feature] = flat_group_id[feature].transform('max')

  df['src_count'] = flat_group_id['src'].transform('count')


  cnt_list = ['cnt_dst','cnt_serv_src', 'cnt_dst_slow', 'cnt_serv_src_slow', 'cnt_dst_conn', 'cnt_serv_src_conn']
  flat_group_id = df.groupby(['src','sub_dst'])
  for feature in cnt_list:
    df[feature] = flat_group_id[feature].transform('max')

  df['dst_count'] = flat_group_id['dst'].transform('count')


  df = flat_group_id.apply(checkprobe)


  df.drop(columns = drop_list, inplace = True)


  global feature_col
  feature_col = [ i for i in df.columns if i not in ['time','src','dst','spt','dpt','label','en_label','out (bytes)','in (bytes)','duration','app','proto']]
  
  if drop :
    df.drop_duplicates(subset=feature_col,keep='first',inplace=True)
  
  return df

In [5]:
df_10 = pd.read_csv('/content/drive/MyDrive/2021_nad_challenge_dataset/training set/1210_firewall.csv')
df_03 = pd.read_csv('/content/drive/MyDrive/2021_nad_challenge_dataset/training set/1203_firewall.csv')
df_16 = pd.read_csv('/content/drive/MyDrive/2021_nad_challenge_dataset/training set/1216_firewall.csv')


df_10 = trans_df(df_10)
df_03 = trans_df(df_03)
df_16 = trans_df(df_16)

df = pd.concat([df_03,df_10,df_16],ignore_index=True,copy=False)


Ensemble model

In [ ]:
class model:
  def __init__(self,features):
    self.lb = LabelEncoder()
    
    self.xgb_params = {
      'tree_method':'gpu_hist',
      'objective':'multi:softprob',
      # 'objective':'multi:softmax',
      'metric':'mlogloss',
      'eval_metric':'mlogloss',
      'max_delta_step':3,
      'max_depth':4,
      'num_class':5
    }
    self.lgb_params = {
      'num_leaves':11,
      'colsample_bytree':0.8,
      'subsample':0.8,
      'max_depth':4,
      'reg_alpha':0.1,
      'reg_lambda':0.1,
      'min_split_gain':0.1,
      'metric':'multi_logloss',
      'objective':'multiclass',
      'num_class':5
    }
    self.cab_params = {
      'iterations': 100,
      'depth': 3,
      'allow_writing_files': False,
      # 'use_best_model':True,
      'objective':'MultiClass',
      'early_stopping_rounds':10,
      'task_type':'GPU',
      'verbose':False
    }
    self.stack_xgb = {
      'tree_method':'gpu_hist',
      # 'objective':'multi:softprob',
      'objective':'multi:softmax',
      'metric':'mlogloss',
      'eval_metric':'mlogloss',
      'max_delta_step':3,
      'max_depth':4,
      'num_class':5
    }
    self.features = features
  def fit(self,train):

    train['en_label'] = self.lb.fit_transform(train.label.values)
 
    train_xgb = xgb.DMatrix(train[self.features],label=train['en_label'])
    train_lgb = lgb.Dataset(train[self.features],label=train['en_label'])
    train_cab = cab.Pool(train[self.features],train['en_label'])

  
    self.bst_xgb = xgb.train(self.xgb_params,dtrain=train_xgb,num_boost_round=100,verbose_eval=False) 
 
    self.bst_lgb = lgb.train(self.lgb_params,train_set=train_lgb,verbose_eval=False)
 
    self.bst_cab = cab.CatBoost(self.cab_params)

    self.bst_cab.fit(train_cab)

    X = np.concatenate((self.bst_xgb.predict(train_xgb),self.bst_lgb.predict(train[self.features]),self.bst_cab.predict(train_cab)),axis=1)

    train_xgb = xgb.DMatrix(X,label=train['en_label'])

    self.bst = xgb.train(self.stack_xgb,dtrain=train_xgb,num_boost_round=100,verbose_eval=False) 
    
  def predict(self,data):

    for i in (set(self.features) - set(data.columns)):
      data[i] = 0
    test_xgb = xgb.DMatrix(data[self.features])
    test_cab = cab.Pool(data[self.features])

    X_test_stack = np.concatenate((self.bst_xgb.predict(test_xgb),self.bst_lgb.predict(data[self.features]),self.bst_cab.predict(test_cab)),axis=1)
  
    X_test_stack = xgb.DMatrix(X_test_stack)

    label = self.bst.predict(X_test_stack)
    
    return self.lb.inverse_transform(label.astype(int))




Model train and save the pickle file

In [ ]:
bst_model = model(feature_col)
bst_model.fit(df)
model_file = open('/content/drive/MyDrive/114_ictfi_x/model.pkl', 'wb')
pickle.dump(bst_model,model_file)
model_file.close()

In [ ]:
model_file = open('/content/drive/MyDrive/114_ictfi_x/model.pkl', 'rb')
bst_model = pickle.load(model_file)

folder = '/content/drive/MyDrive/2021_nad_challenge_dataset/testing set/'

for root, dirs, files in os.walk(folder):
  for f in files:
    if f.endswith('.csv'):
      test_origin = pd.read_csv(os.path.join(root, f))
      test_df = test_origin.copy()
      test_df = trans_df(test_df,drop=False)
      label = bst_model.predict(test_df)
      test_origin['label'] = label
      del test_df
      gc.collect()
      print(test_origin.label.value_counts())
      test_origin.to_csv(os.path.join('/content/drive/MyDrive/114_ictfi_x/', '114_ictfi_x_'+f),index=0)

Normal        2118678
DDOS-smurf       2141
Name: label, dtype: int64
Normal                3600977
DDOS-smurf                119
Probing-Port sweep         81
Probing-IP sweep            9
Name: label, dtype: int64
Normal              2049173
Probing-IP sweep        928
Probing-Nmap            496
DDOS-smurf              113
Name: label, dtype: int64
Normal                5509470
Probing-IP sweep         6818
DDOS-smurf               1158
Probing-Port sweep        369
Name: label, dtype: int64
